# Backup - Restore with Kasten (a Veeam Company)
created by Stephan.Koch@hpe.com

used best of breed components
- from SuSE/Rancher: Operating System and Kubernetes Distro
- from IBM/Red Hat: Automation Engine: Ansible
- from VMware: Hypervisor, Cloud Provider and CSI
- from Veeam/Kasten: Backup Software
- from Microsoft: Azure Blob Storage
- from HPE: Blech and Consultant

## First create an K3S cluster

In [50]:

cd ~/notebooks/k8s/storage/
rm -rf kastendemo/*yml kastendemo/.inv* kastendemo/node-token kastendemo/vsphere* kastendemo/k3s*
mkdir kastendemo
cd kastendemo
pwd
ls


mkdir: cannot create directory 'kastendemo': File exists
/home/skoch/notebooks/k8s/storage/kastendemo
packer_id_rsa


#### create VMs with OS Suse Linux Micro 5.0 

In [57]:
cd ~/notebooks/k8s/storage/kastendemo
export ANSIBLE_VAULT_PASSWORD_FILE=~/notebooks/k8s/.vault_password_file
cat << 'EOF' > .create_vms.yml 
---
- hosts: localhost
  gather_facts: no


  vars:
    ip_addr: "50"
    amount: "3"

    vcenter_pass: !vault |
        $ANSIBLE_VAULT;1.1;AES256
        65363864613935346332643061326331373330656238356631373661393266633665383839353366
        3365663565626136636430353431316433636535336330640a346536313330656235636364663637
        34333634313939393266666261383835353266373865386632626631666463366535346563396138
        3866356538623064380a643830653330653530636137616537636131636631313565303236623839
        3063
    vcenter_server: "vc.container.demo.local"
    vcenter_user: "skoch@demo.local"
    datacenter_name: "ctcbbn"
    cluster_name: "Cluster"
    template_name: "template-sle-micro5-base"
    prefix_ip: "10.1.35"
    endip: "{{ ip_addr|int + amount|int -1 }}"

  tasks:
    - name: Clone the template
      community.vmware.vmware_guest:
        hostname: "{{ vcenter_server }}"
        username: "{{ vcenter_user }}"
        password: "{{ vcenter_pass }}"
        validate_certs: False
        name: "k3s-{{ item }}"
        template: "{{ template_name }}"
        datacenter: "{{ datacenter_name }}"
        folder: /k3sdemo
        cluster: "{{ cluster_name }}"
        datastore: "DS3"
        hardware:
          memory_mb: 2048
          num_cpus: 1
        networks:
        - name: DPortGroup35
          connected: yes
          start_connected: yes
          ip: "{{ prefix_ip }}.{{ item }}"
          netmask: 255.255.255.0
          gateway: 10.1.35.1
          type: static
          dns_servers: 10.1.20.5
        disk:
          - autoselect_datastore: no
            datastore: "DS3"
            size_gb: 30
            type: thin
        customization:
          hostname: "k3s-{{ item }}"
          domain: container.demo.local
          dns_servers:
          - 10.1.20.5
          - 10.1.20.6
        state: poweredon
        wait_for_ip_address: no
      with_sequence: start={{ip_addr}} end={{endip}} stride=1
      delegate_to: localhost

    - name: Creating hosts file for step 2
      copy:
        dest: "~/notebooks/k8s/storage/kastendemo/.inventory1"
        content: |
          [master]
          {{ prefix_ip }}.{{ip_addr}}

          [nodes]

    - name: Add all nodes
      ansible.builtin.lineinfile:
        path: "~/notebooks/k8s/storage/kastendemo/.inventory1"
        line: "{{ prefix_ip }}.{{ item }}"
        create: yes
      with_sequence: start={{ip_addr|int + 1}} end={{endip}} stride=1
EOF
ansible-playbook .create_vms.yml


PLAY [localhost] ***************************************************************

TASK [Clone the template] ******************************************************
ok: [localhost -> localhost] => (item=50)
ok: [localhost -> localhost] => (item=51)
ok: [localhost -> localhost] => (item=52)

TASK [Creating hosts file for step 2] ******************************************
changed: [localhost]

TASK [Add all nodes] ***********************************************************
changed: [localhost] => (item=51)
changed: [localhost] => (item=52)

PLAY RECAP *********************************************************************
localhost                  : ok=3    changed=2    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



#### create K3S cluster on above VMs

In [53]:
#check if up
ssh -o StrictHostKeyChecking=no -i packer_id_rsa bot@10.1.35.52 date

Thu Apr  8 12:33:06 UTC 2021


In [58]:
cat << 'EOF' > .create_K3S.yml 

- hosts: all
  gather_facts: no
  remote_user: bot
  vars:
    ansible_ssh_private_key_file: ./packer_id_rsa
    ansible_host_key_checking: false
  tasks:
  - name: Wait for port ssh to become open on the host
    wait_for:
      port: 22
      delay: 10

- hosts: master
  gather_facts: no
  remote_user: bot
  become: yes
  become_method: sudo
  vars:
    ansible_ssh_private_key_file: ./packer_id_rsa
    ansible_host_key_checking: false


# use disable cloud provider for be able to set ProviderID for out-of-tree vsphere-CSI
  tasks:
    - name: install k3s on master node
      shell: "curl -sfL https://get.k3s.io |  sh -s - --disable-cloud-controller"
      args:
        warn: false
        creates: /usr/local/bin/k3s-uninstall.sh

# Retrieve the server token and place it here
    - name: fetch the server token
      fetch:
        src: /var/lib/rancher/k3s/server/node-token
        dest: "./node-token"
        flat: yes


# install k3s agent on nodes
- hosts: nodes
  gather_facts: no
  remote_user: bot
  become: yes
  become_method: sudo
  vars:
    ansible_ssh_private_key_file: packer_id_rsa
    file: .inventory1
  tasks:
    - set_fact:
        k3s_token: "{{ lookup('file','./node-token')  }}"
        k3s_url: "https://{{item}}:6443"
      with_lines: cat  {{ file }} | head -2 | tail -1
        #k3s_url: "{{ k3s_url }}"
    - name: install k3s on nodes
      shell: "curl -sfL https://get.k3s.io | K3S_URL={{ k3s_url }} K3S_TOKEN={{ k3s_token }} sh - "
      args:
        warn: false
        creates: /usr/local/bin/k3s-agent-uninstall.sh

# show results
- hosts: master
  gather_facts: no
  remote_user: bot
  become: yes
  become_method: sudo

  tasks:
  - name: kubectl get nodes
    shell: "/usr/local/bin/k3s kubectl get nodes"
    register: kctl

  - debug: msg={{ kctl.stdout_lines }}

EOF
export ANSIBLE_HOST_KEY_CHECKING=False
ansible-playbook  -i .inventory1   .create_K3S.yml


PLAY [all] *********************************************************************

TASK [Wait for port ssh to become open on the host] ****************************
[WARNING]: Platform linux on host 10.1.35.50 is using the discovered Python
interpreter at /usr/bin/python3.6, but future installation of another Python
interpreter could change the meaning of that path. See https://docs.ansible.com
/ansible/2.10/reference_appendices/interpreter_discovery.html for more
information.
ok: [10.1.35.50]
[WARNING]: Platform linux on host 10.1.35.52 is using the discovered Python
interpreter at /usr/bin/python3.6, but future installation of another Python
interpreter could change the meaning of that path. See https://docs.ansible.com
/ansible/2.10/reference_appendices/interpreter_discovery.html for more
information.
ok: [10.1.35.52]
[WARNING]: Platform linux on host 10.1.35.51 is using the discovered Python
interpreter at /usr/bin/python3.6, but future installation of another Python
interpreter cou

#### fetch kubeconfig file

In [59]:
cat << 'EOF' > .fetch_k8sconfig.yml 
- hosts: master
  gather_facts: no
  remote_user: bot
  become: yes
  become_method: sudo
  vars:
    ansible_ssh_private_key_file: ./packer_id_rsa
    ansible_host_key_checking: false
    file: .inventory1

  tasks:
# Retrieve the server token and place it here
    - name: fetch the server token
      ansible.builtin.fetch:
        src: /etc/rancher/k3s/k3s.yaml
        dest: "./k3s-kubeconfig"
        flat: yes

        
    - set_fact:
        master: "{{item}}"
      with_lines: cat  {{ file }} | head -2 | tail -1
      
    - name: replace 127.0.0.1 with master ip
      replace:
        path:  k3s-kubeconfig 
        regexp: '127.0.0.1'
        replace: '{{ master }}'
        backup: no  
      delegate_to: localhost  
        

EOF
ansible-playbook  -i .inventory1   .fetch_k8sconfig.yml

chmod 600 k3s-kubeconfig
export KUBECONFIG=$PWD/k3s-kubeconfig
kubectl get nodes


PLAY [master] ******************************************************************

TASK [fetch the server token] **************************************************
changed: [10.1.35.50]

TASK [set_fact] ****************************************************************
ok: [10.1.35.50] => (item=10.1.35.50)

TASK [replace 127.0.0.1 with master ip] ****************************************
changed: [10.1.35.50]

PLAY RECAP *********************************************************************
10.1.35.50                 : ok=3    changed=2    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   

NAME     STATUS   ROLES                  AGE   VERSION
k3s-50   Ready    control-plane,master   45s   v1.20.5+k3s1
k3s-51   Ready    <none>                 32s   v1.20.5+k3s1
k3s-52   Ready    <none>                 32s   v1.20.5+k3s1


In [60]:
kubectl get no

NAME     STATUS   ROLES                  AGE   VERSION
k3s-50   Ready    control-plane,master   54s   v1.20.5+k3s1
k3s-51   Ready    <none>                 41s   v1.20.5+k3s1
k3s-52   Ready    <none>                 41s   v1.20.5+k3s1


## Install vSphere- Cloud Provider and Container Storage Interface


In [61]:
#ProviderID should NOT be set; k8s only allow the change from "" 
kubectl describe node| grep -i ProviderID

: 1

##### Update ProviderIDs in K8S and set disk.enableUUID=1 for VMs
On a machine with govc, go jq, and kubectl
- https://github.com/vmware/govmomi/tree/master/govc
- govc is a vSphere CLI built on top of govmomi.
- govmomi: Go library for the VMware vSphere API

In [62]:
#!/bin/bash

export GOVC_USERNAME='skoch@demo.local'
export GOVC_INSECURE=1
export GOVC_PASSWORD=$(echo "UmF0aW5nZW4uMTIzCg=="|base64 -d)
export GOVC_URL='vc.container.demo.local'
DATACENTER='ctcbbn'
FOLDER='k3sdemo'
# In my case I'm using a prefix for the VM's, so grep'ing is necessary.
# You can remove it if the folder you are using only contains the machines you need.
VM_PREFIX='/k3s-'
IFS=$'\n'
for vm in $(govc ls "/$DATACENTER/vm/$FOLDER" | grep $VM_PREFIX); do
  MACHINE_INFO=$(govc vm.info -json -dc=$DATACENTER -vm.ipath="/$vm" -e=true)
  # My VMs are created on vmware with upper case names, so I need to edit the names with awk
  VM_NAME=$(jq -r ' .VirtualMachines[] | .Name' <<< $MACHINE_INFO | awk '{print tolower($0)}')
  # UUIDs come in lowercase, upper case then
  VM_UUID=$( jq -r ' .VirtualMachines[] | .Config.Uuid' <<< $MACHINE_INFO | awk '{print toupper($0)}')
  echo "Patching $VM_NAME with UUID:$VM_UUID"
  # This is done using dry-run to avoid possible mistakes, remove when you are confident you got everything right.
  kubectl patch node $VM_NAME -p "{\"spec\":{\"providerID\":\"vsphere://$VM_UUID\"}}"
done


Patching k3s-52 with UUID:421ACAE7-F241-FAFF-7276-6B78AA9F8BC1
node/k3s-52 patched
Patching k3s-51 with UUID:421AD3BE-231E-B7CB-B29B-43286FA779A1
node/k3s-51 patched
Patching k3s-50 with UUID:421A6F75-B095-FC0F-1F37-B91FB14ED18A
node/k3s-50 patched


####executed externally 

In [63]:
kubectl describe node| grep -i ProviderID

ProviderID:                   vsphere://421ACAE7-F241-FAFF-7276-6B78AA9F8BC1
ProviderID:                   vsphere://421AD3BE-231E-B7CB-B29B-43286FA779A1
ProviderID:                   vsphere://421A6F75-B095-FC0F-1F37-B91FB14ED18A


have to set disk.enableUUID=1 for all vms

In [64]:

for vm in $(govc ls "/$DATACENTER/vm/$FOLDER" | grep $VM_PREFIX); do
    MACHINE_INFO=$(govc vm.info -json -dc=$DATACENTER -vm.ipath="/$vm" -e=true)
    VM_NAME=$(jq -r ' .VirtualMachines[] | .Name' <<< $MACHINE_INFO | awk '{print tolower($0)}')
    echo "Patching $VM_NAME"
    govc vm.change -e="disk.enableUUID=1" -vm=$VM_NAME
done

Patching k3s-52
Patching k3s-51
Patching k3s-50


#### Install CP and CSI with helm

In [65]:
#https://github.com/stefanvangastel/vsphere-cpi-csi-helm
git clone https://github.com/stefanvangastel/vsphere-cpi-csi-helm.git

Cloning into 'vsphere-cpi-csi-helm'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 165 (delta 35), reused 50 (delta 18), pack-reused 78
Receiving objects: 100% (165/165), 64.20 KiB | 0 bytes/s, done.
Resolving deltas: 100% (63/63), done.


In [66]:
cd vsphere-cpi-csi-helm

helm install vsphere-cpi-csi \
     --namespace kube-system \
     ./charts/vsphere-cpi-csi/v2.1.0 \
     --set vcenter.host=vc.container.demo.local \
     --set vcenter.username=skoch@demo.local \
     --set vcenter.password=$(echo "UmF0aW5nZW4uMTIzCg=="|base64 -d) \
     --set insecure-flag=true \
     --set vcenter.datacenter=ctcbbn

NAME: vsphere-cpi-csi
LAST DEPLOYED: Thu Apr  8 14:42:06 2021
NAMESPACE: kube-system
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [67]:
kubectl get pod -n kube-system

NAME                                      READY   STATUS              RESTARTS   AGE
coredns-854c77959c-xlh4k                  1/1     Running             0          97s
local-path-provisioner-5ff76fc89d-6gwpt   1/1     Running             0          97s
metrics-server-86cbb8457f-rlrss           1/1     Running             0          97s
helm-install-traefik-pmx79                0/1     Completed           0          97s
svclb-traefik-9thf9                       2/2     Running             0          77s
svclb-traefik-zvhqx                       2/2     Running             0          77s
svclb-traefik-c27rn                       2/2     Running             0          77s
traefik-6f9cbd9bd4-nwcrz                  1/1     Running             0          77s
vsphere-csi-controller-789b8f4956-rx596   0/6     Pending             0          4s
vsphere-csi-node-9tkhb                    0/3     ContainerCreating   0          4s
vsphere-csi-node-dfngv                    0/3     ContainerCreating

#### patching deployment because K3S label nodes as "control-plane" and not "controlplane"

In [68]:
kubens kube-system
kubectl patch deployment vsphere-csi-controller --type=json -p='[{"op": "remove", "path": "/spec/template/spec/nodeSelector"}]'
kubectl patch --type merge deployment vsphere-csi-controller -p '{ "spec": { "template": { "spec": { "nodeSelector": { "node-role.kubernetes.io/control-plane": "true" } } } } }'

Context "default" modified.
Active namespace is "kube-system".
deployment.apps/vsphere-csi-controller patched
deployment.apps/vsphere-csi-controller patched


delete nodeselector node-role.kubernetes.io/controlplane "without "-""
via kubectl edit deployment vsphere-csi-controller

In [77]:
kubectl get pod -n kube-system

NAME                                      READY   STATUS      RESTARTS   AGE
coredns-854c77959c-xlh4k                  1/1     Running     0          2m52s
local-path-provisioner-5ff76fc89d-6gwpt   1/1     Running     0          2m52s
metrics-server-86cbb8457f-rlrss           1/1     Running     0          2m52s
helm-install-traefik-pmx79                0/1     Completed   0          2m52s
svclb-traefik-9thf9                       2/2     Running     0          2m32s
svclb-traefik-zvhqx                       2/2     Running     0          2m32s
svclb-traefik-c27rn                       2/2     Running     0          2m32s
traefik-6f9cbd9bd4-nwcrz                  1/1     Running     0          2m32s
vsphere-csi-node-dfngv                    3/3     Running     0          79s
vsphere-csi-node-9tkhb                    3/3     Running     0          79s
vsphere-csi-node-srtr2                    3/3     Running     0          79s
vsphere-csi-controller-74c8855867-fjm2r   6/6     Running   

#### check if sc and csidriver is defined

In [73]:
kubectl get sc,csidrivers

NAME                                                PROVISIONER              RECLAIMPOLICY   VOLUMEBINDINGMODE      ALLOWVOLUMEEXPANSION   AGE
storageclass.storage.k8s.io/local-path (default)    rancher.io/local-path    Delete          WaitForFirstConsumer   false                  2m45s
storageclass.storage.k8s.io/vsphere-csi (default)   csi.vsphere.vmware.com   Delete          Immediate              false                  60s

NAME                                              ATTACHREQUIRED   PODINFOONMOUNT   MODES        AGE
csidriver.storage.k8s.io/csi.vsphere.vmware.com   true             false            Persistent   60s


#### check if csi working

In [74]:
cat << 'EOF' | kubectl apply -f -
---
kind: PersistentVolumeClaim
apiVersion: v1
metadata:
  name: csi-pvc1
  labels:
    cluster: ocp46
  annotations:
    volume.beta.kubernetes.io/storage-class: vsphere-csi
spec:
  accessModes:
  - ReadWriteOnce
  resources:
    requests:
      storage: 1400Mi
EOF


persistentvolumeclaim/csi-pvc1 created


In [75]:
kubectl get pvc

NAME       STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
csi-pvc1   Bound    pvc-2a3f4465-a1b0-41b5-a367-99f89d288999   2Gi        RWO            vsphere-csi    4s


In [76]:
kubectl delete pvc/csi-pvc1

persistentvolumeclaim "csi-pvc1" deleted


## Install Kasten

In [78]:
kubectl get sc

NAME                    PROVISIONER              RECLAIMPOLICY   VOLUMEBINDINGMODE      ALLOWVOLUMEEXPANSION   AGE
local-path (default)    rancher.io/local-path    Delete          WaitForFirstConsumer   false                  3m11s
vsphere-csi (default)   csi.vsphere.vmware.com   Delete          Immediate              false                  86s


In [79]:
helm repo add kasten https://charts.kasten.io/
helm repo update

"kasten" has been added to your repositories
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "hpe-storage" chart repository
...Successfully got an update from the "kasten" chart repository
...Successfully got an update from the "cormachogan" chart repository
...Successfully got an update from the "halkeye" chart repository
...Successfully got an update from the "harbor" chart repository
...Unable to get an update from the "stable" chart repository (https://kubernetes-charts.storage.googleapis.com):
	failed to fetch https://kubernetes-charts.storage.googleapis.com/index.yaml : 403 Forbidden
...Successfully got an update from the "bitnami" chart repository
Update Complete. ⎈ Happy Helming!⎈ 


In [80]:
kubectl create ns kasten-io
helm install k10 kasten/k10 --namespace=kasten-io --set global.persistence.storageClass=vsphere-csi 

namespace/kasten-io created
NAME: k10
LAST DEPLOYED: Thu Apr  8 14:43:44 2021
NAMESPACE: kasten-io
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Thank you for installing Kasten’s K10 Data Management Platform!

Documentation can be found at https://docs.kasten.io/.

How to access the K10 Dashboard:



The K10 dashboard is not exposed externally. To establish a connection to it use the following `kubectl` command:

`kubectl --namespace kasten-io port-forward service/gateway 8080:8000`

The Kasten dashboard will be available at: `http://127.0.0.1:8080/k10/#/`


In [83]:
kubectl config set-context --current --namespace=kasten-io
kubectl get pod,pvc

Context "default" modified.
NAME                                      READY   STATUS    RESTARTS   AGE
pod/dashboardbff-svc-6556497b9-r8wl2      1/1     Running   0          118s
pod/logging-svc-d4c959f77-ksgf9           1/1     Running   0          116s
pod/jobs-svc-5c5fbb6ddb-k54vb             1/1     Running   0          118s
pod/state-svc-6cd69b76b9-5rs7x            1/1     Running   0          116s
pod/auth-svc-89779cf75-6qjq4              1/1     Running   0          116s
pod/aggregatedapis-svc-65cdccd9bd-l988b   1/1     Running   0          116s
pod/config-svc-6b4dd8b699-52t4j           1/1     Running   0          116s
pod/crypto-svc-84454c9c7c-kz6w4           1/1     Running   0          115s
pod/metering-svc-6fd6f86cb-cfm48          1/1     Running   0          116s
pod/frontend-svc-6f9d98bc9c-vxz8h         1/1     Running   0          117s
pod/executor-svc-844cfdc565-2v5rq         2/2     Running   0          118s
pod/kanister-svc-6d95cbcd5-hpdhm          1/1     Running   0

In [ ]:
# on master node as root
#k3s kubectl --namespace kasten-io port-forward --address 0.0.0.0 service/gateway 8080:8000 

![title](pictures/Welcome.png)

#### define external Storage Location (in my Case Azure Blob)

![AZBlob](pictures/Backup-Destination-AZ.png)

In [25]:
kubectl get profile az-stephan -o yaml| kubectl neat

apiVersion: config.kio.kasten.io/v1alpha1
kind: Profile
metadata:
  name: az-stephan
  namespace: kasten-io
spec:
  locationSpec:
    credential:
      secret:
        apiVersion: v1
        kind: secret
        name: k10secret-bpvrb
        namespace: kasten-io
      secretType: AzStorageAccount
    objectStore:
      name: kasten-demo
      objectStoreType: AZ
      path: k10/9687a54f-87ed-4b38-82ca-d88eac0f965e/migration
      pathType: Directory
      region: West Europe
    type: ObjectStore
  type: Location



#### define an Infrastructure Provider

![InfraProvider](pictures/Infra-Provider.png)

In [34]:
kubectl get profile vsphere-ctc -o yaml| kubectl neat

apiVersion: config.kio.kasten.io/v1alpha1
kind: Profile
metadata:
  name: vsphere-ctc
  namespace: kasten-io
spec:
  infra:
    credential:
      secret:
        apiVersion: v1
        kind: secret
        name: k10secret-qp46q
        namespace: kasten-io
      secretType: VSphereKey
    type: VSphere
    vsphere:
      serverAddress: vc.container.demo.local
  type: Infra



#### enable K10 DR

![title](pictures/Enable-K10-DR.png)

In [41]:
kubectl get secret k10-dr-secret  -o yaml| kubectl neat


apiVersion: v1
data:
  key: a2FzdGVuMXNjODhs
kind: Secret
metadata:
  name: k10-dr-secret
  namespace: kasten-io
type: Opaque



![title](pictures/Enable-K10-DR-2.png)

In [84]:
kubectl api-resources --verbs=list | grep -i kasten

backupactions                                  actions.kio.kasten.io          true         BackupAction
backupclusteractions                           actions.kio.kasten.io          false        BackupClusterAction
exportactions                                  actions.kio.kasten.io          true         ExportAction
importactions                                  actions.kio.kasten.io          true         ImportAction
restoreactions                                 actions.kio.kasten.io          true         RestoreAction
restoreclusteractions                          actions.kio.kasten.io          false        RestoreClusterAction
retireactions                                  actions.kio.kasten.io          false        RetireAction
runactions                                     actions.kio.kasten.io          false        RunAction
applications                                   apps.kio.kasten.io             true         Application
clusterrestorepoints                           apps.

In [85]:

kubectl get secret

NAME                            TYPE                                  DATA   AGE
default-token-x69qn             kubernetes.io/service-account-token   3      5m41s
kopia-tls-cert                  Opaque                                1      5m40s
k10-trial-license               Opaque                                1      5m40s
kopia-tls-key                   Opaque                                1      5m40s
k10-license                     Opaque                                1      5m40s
prometheus-server-token-lqmdd   kubernetes.io/service-account-token   3      5m40s
k10-k10-token-w2c5j             kubernetes.io/service-account-token   3      5m40s
sh.helm.release.v1.k10.v1       helm.sh/release.v1                    1      5m40s
k10-cluster-passphrase          Opaque                                1      4m16s
k10secret-7qqtf                 Opaque                                2      2m2s
k10secret-gz7x8                 Opaque                                2      95s
k10-dr-se

#### create backup for cluster-scoped resources with export

## for DR purpose save some inmportant values

In [86]:
 #Extract UUID of the `default` namespace for DR porpuse
 kubectl get namespace default -ojsonpath="{.metadata.uid}{'\n'}"

840916bd-40e0-4f19-8dee-631bd857c2ef


In [87]:
kubectl get secret  k10-dr-secret -o yaml --namespace kasten-io |kubectl neat
echo "---"
kubectl get secret  k10-dr-secret -o json --namespace kasten-io | jq -r .data.key|base64 -d

apiVersion: v1
data:
  key: a2FzdGVuMXNjODhs
kind: Secret
metadata:
  name: k10-dr-secret
  namespace: kasten-io
type: Opaque

---
kasten1sc88l

In [88]:
kubectl get profile

NAME          STATUS
az-stephan    Success
vsphere-ctc   Success


In [89]:
kubectl get profile az-stephan -o yaml | kubectl neat

apiVersion: config.kio.kasten.io/v1alpha1
kind: Profile
metadata:
  name: az-stephan
  namespace: kasten-io
spec:
  locationSpec:
    credential:
      secret:
        apiVersion: v1
        kind: secret
        name: k10secret-7qqtf
        namespace: kasten-io
      secretType: AzStorageAccount
    objectStore:
      name: kasten-demo
      objectStoreType: AZ
      path: k10/840916bd-40e0-4f19-8dee-631bd857c2ef/migration
      pathType: Directory
      region: West Europe
    type: ObjectStore
  type: Location



In [91]:
kubectl get profile vsphere-ctc -o yaml | kubectl neat

apiVersion: config.kio.kasten.io/v1alpha1
kind: Profile
metadata:
  name: vsphere-ctc
  namespace: kasten-io
spec:
  infra:
    credential:
      secret:
        apiVersion: v1
        kind: secret
        name: k10secret-gz7x8
        namespace: kasten-io
      secretType: VSphereKey
    type: VSphere
    vsphere:
      serverAddress: vc.container.demo.local
  type: Infra



### my backup policies for reference and DR

In [112]:
kubectl get policies -n kasten-io -o yaml| kubectl neat

apiVersion: v1
items:
- apiVersion: config.kio.kasten.io/v1alpha1
  kind: Policy
  metadata:
    name: k10-disaster-recovery-policy
    namespace: kasten-io
  spec:
    actions:
    - action: backup
      backupParameters:
        profile:
          name: az-stephan
          namespace: kasten-io
    frequency: '@hourly'
    retention:
      daily: 1
      hourly: 4
      monthly: 1
      weekly: 1
      yearly: 1
    selector:
      matchExpressions:
      - key: k10.kasten.io/appNamespace
        operator: In
        values:
        - kasten-io
kind: List
metadata:
  resourceVersion: ""
  selfLink: ""



# Install Demo Application Wordpress with MariaDB

In [92]:
kubectl create ns my-wordpress
kubectl config set-context --current --namespace=my-wordpress

namespace/my-wordpress created
Context "default" modified.


In [93]:
helm repo add bitnami https://charts.bitnami.com/bitnami

"bitnami" has been added to your repositories


In [94]:
helm install my-wordpress \
  --set wordpressUsername=admin \
  --set wordpressPassword=password \
  --set mariadb.auth.rootPassword=secretpassword \
  --set mariadb.auth.password=notsosecret \
  --set global.storageClass=vsphere-csi \
  --set service.type=NodePort \
    bitnami/wordpress

NAME: my-wordpress
LAST DEPLOYED: Thu Apr  8 15:30:51 2021
NAMESPACE: my-wordpress
STATUS: deployed
REVISION: 1
NOTES:
** Please be patient while the chart is being deployed **

Your WordPress site can be accessed through the following DNS name from within your cluster:

    my-wordpress.my-wordpress.svc.cluster.local (port 80)

To access your WordPress site from outside the cluster follow the steps below:

1. Get the WordPress URL by running these commands:

   export NODE_PORT=$(kubectl get --namespace my-wordpress -o jsonpath="{.spec.ports[0].nodePort}" services my-wordpress)
   export NODE_IP=$(kubectl get nodes --namespace my-wordpress -o jsonpath="{.items[0].status.addresses[0].address}")
   echo "WordPress URL: http://$NODE_IP:$NODE_PORT/"
   echo "WordPress Admin URL: http://$NODE_IP:$NODE_PORT/admin"

2. Open a browser and access WordPress using the obtained URL.

3. Login with the following credentials below to see your blog:

  echo Username: admin
  echo Password: $(kubectl

In [106]:
kubectl get all

NAME                               READY   STATUS    RESTARTS   AGE
pod/my-wordpress-mariadb-0         1/1     Running   0          14m
pod/my-wordpress-d887bbd78-dmlhn   1/1     Running   0          14m

NAME                           TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)                      AGE
service/my-wordpress-mariadb   ClusterIP   10.43.20.91    <none>        3306/TCP                     14m
service/my-wordpress           NodePort    10.43.229.92   <none>        80:31195/TCP,443:30983/TCP   14m

NAME                           READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/my-wordpress   1/1     1            1           14m

NAME                                     DESIRED   CURRENT   READY   AGE
replicaset.apps/my-wordpress-d887bbd78   1         1         1       14m

NAME                                    READY   AGE
statefulset.apps/my-wordpress-mariadb   1/1     14m


In [105]:
# in case of docker limit reached: do an docker login and use the created .docker/config.json for increasing 
cd ~/notebooks/k8s/storage/kastendemo/
kubectl create secret generic regcred \
    --from-file=.dockerconfigjson=config.json \
    --type=kubernetes.io/dockerconfigjson

secret/regcred created


In [108]:
kubectl get pvc

NAME                          STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
my-wordpress                  Bound    pvc-56ca3a54-c7fd-41c9-b157-8e1fac27fc4c   10Gi       RWO            vsphere-csi    15m
data-my-wordpress-mariadb-0   Bound    pvc-1050b7dd-e7e9-4f22-97bc-4a4763ca7319   8Gi        RWO            vsphere-csi    15m


### do some changes on Wordpress

login with: http://10.1.35.51:31195/login


#### create Backup in Kasten


![wordpressBackup](pictures/my-wordpress-backup.png)

In [116]:
kubectl get policies my-wordpress-backup -n kasten-io -o yaml| kubectl neat

apiVersion: config.kio.kasten.io/v1alpha1
kind: Policy
metadata:
  name: my-wordpress-backup
  namespace: kasten-io
spec:
  actions:
  - action: backup
    backupParameters:
      profile:
        name: az-stephan
        namespace: kasten-io
  - action: export
    exportParameters:
      exportData:
        enabled: true
      frequency: '@weekly'
      migrationToken:
        name: my-wordpress-backup-migration-token-q5d6v
        namespace: kasten-io
      profile:
        name: az-stephan
        namespace: kasten-io
      receiveString: bIzAPpoanmEpnjszvlUynfclRGkuPneFySzSpUwzW6kbL+x7h3SybL+aZuOrkKg6BxfWzKuNGWik9SNd1g6xyGY0+AYfLO+bYbay8eWagcya56Fh53Acb1mo6pHIA70rT4EKuAoOkeJJsuvRtK3Sw0mnMsHTxQIVp1nSIEidip3E0YMIAwLY2mDV6yKCbUs2o+dyYc3z/mkbw1wMLCPKoNq5TDDqQDuzuTIdooWMboUeuIqLcm6OZMYM9aSZxcC4NMiNOayPdZxwSXj2p0gksifK9eKDaujY1AuMQys1W2RnXkTeq2Bwrjafuc3H9t1i1ldiZJhahHEDmcRowHypHfNatpFUeRIT79U5xNG9p7fyw0SEptrdt3tv8xfyw3t7sNHiLJxB3veiH34nn5AfCyl56LCaQhVSTlLeqB7m
  frequency: '@weekly'
  re

### Be destructive !

In [421]:
kubectl delete ns my-wordpress

namespace "my-wordpress" deleted


In [117]:
kubectl get all,pvc -n my-wordpress

No resources found in my-wordpress namespace.


### Restore now

![wordpressRestore](pictures/Restore-wordpress.png)

##### IF restore through GUI failed
(failed with ver 3.0.10 but not with 3.0.11)

```ERROR MESSAGES
  Job failed to be executed
 Failed to restore spec artifacts
 Service "my-wordpress-mariadb" is invalid: spec.clusterIPs: Invalid value: []string{"10.43.124.158"}: must be empty when clusterIP is empty
 ```

In [424]:
#output after restore!
kubectl get svc -o yaml| grep -i clusterip

    clusterIP: 10.43.62.136
    clusterIPs:
    type: ClusterIP
    clusterIP: 10.43.61.180
    clusterIPs:


#### either install fresh and do an restore OR define services manually and do the restore

In [367]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
items:
- apiVersion: v1
  kind: Service
  metadata:
    annotations:
      meta.helm.sh/release-name: my-wordpress
      meta.helm.sh/release-namespace: my-wordpress
    labels:
      app.kubernetes.io/component: primary
      app.kubernetes.io/instance: my-wordpress
      app.kubernetes.io/managed-by: Helm
      app.kubernetes.io/name: mariadb
      helm.sh/chart: mariadb-9.3.6
    name: my-wordpress-mariadb
    namespace: my-wordpress
  spec:
    clusterIP: 
    clusterIPs:
    
    ports:
    - name: mysql
      port: 3306
      targetPort: mysql
    selector:
      app.kubernetes.io/component: primary
      app.kubernetes.io/instance: my-wordpress
      app.kubernetes.io/name: mariadb
- apiVersion: v1
  kind: Service
  metadata:
    annotations:
      meta.helm.sh/release-name: my-wordpress
      meta.helm.sh/release-namespace: my-wordpress
    labels:
      app.kubernetes.io/instance: my-wordpress
      app.kubernetes.io/managed-by: Helm
      app.kubernetes.io/name: wordpress
      helm.sh/chart: wordpress-10.8.0
    name: my-wordpress
    namespace: my-wordpress
  spec:
    clusterIP: 
    clusterIPs:
    
    ports:
    - name: http
      nodePort: 32183
      port: 80
      targetPort: http
    - name: https
      nodePort: 31673
      port: 443
      targetPort: https
    selector:
      app.kubernetes.io/instance: my-wordpress
      app.kubernetes.io/name: wordpress
    type: NodePort
kind: List
metadata:
  resourceVersion: ""
  selfLink: ""
EOF

service/my-wordpress-mariadb created
service/my-wordpress created


##### restore now ....... succeeded

In [119]:
kubectl get all,pvc

NAME                               READY   STATUS    RESTARTS   AGE
pod/my-wordpress-mariadb-0         1/1     Running   0          7m51s
pod/my-wordpress-d887bbd78-hv4k7   1/1     Running   1          7m51s

NAME                           TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)                      AGE
service/my-wordpress-mariadb   ClusterIP   10.43.162.228   <none>        3306/TCP                     7m54s
service/my-wordpress           NodePort    10.43.38.112    <none>        80:32665/TCP,443:30314/TCP   7m54s

NAME                           READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/my-wordpress   1/1     1            1           7m51s

NAME                                     DESIRED   CURRENT   READY   AGE
replicaset.apps/my-wordpress-d887bbd78   1         1         1       7m51s

NAME                                    READY   AGE
statefulset.apps/my-wordpress-mariadb   1/1     7m52s

NAME                                                STATUS   VOLUME      

### now be really destructive and delete the whole cluster/VMs

### reinstall cluster like above, reinstall vsphere cp+csi
follow the steps above until installing Kasten

In [446]:
helm repo add kasten https://charts.kasten.io/
helm repo update

"kasten" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "kasten" chart repository
...Successfully got an update from the "bitnami" chart repository
...Successfully got an update from the "stable" chart repository
Update Complete. ⎈Happy Helming!⎈


In [447]:
kubectl create ns kasten-io
kubens kasten-io

namespace/kasten-io created
Context "default" modified.
Active namespace is "kasten-io".


In [448]:
#reinstall Kasten
kubectl create ns kasten-io
helm install k10 kasten/k10 --namespace=kasten-io --set global.persistence.storageClass=vsphere-csi 

Error from server (AlreadyExists): namespaces "kasten-io" already exists
NAME: k10
LAST DEPLOYED: Wed Apr  7 15:22:20 2021
NAMESPACE: kasten-io
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Thank you for installing Kasten’s K10 Data Management Platform!

Documentation can be found at https://docs.kasten.io/.

How to access the K10 Dashboard:



The K10 dashboard is not exposed externally. To establish a connection to it use the following `kubectl` command:

`kubectl --namespace kasten-io port-forward service/gateway 8080:8000`

The Kasten dashboard will be available at: `http://127.0.0.1:8080/k10/#/`


#### create location profile (in my case my Azure Blob)

In [450]:
cat << 'EOF' | kubectl apply -f -
apiVersion: config.kio.kasten.io/v1alpha1
kind: Profile
metadata:
  name: az-stephan
  namespace: kasten-io
spec:
  locationSpec:
    credential:
      secret:
        apiVersion: v1
        kind: secret
        name: k10secret-dm9zg
        namespace: kasten-io
      secretType: AzStorageAccount
    objectStore:
      name: kasten-demo
      objectStoreType: AZ
      path: k10/893bbee8-f8d3-4d89-a908-e105f893ad58/migration
      pathType: Directory
      region: West Europe
    type: ObjectStore
  type: Location
EOF

profile.config.kio.kasten.io/az-stephan created


In [451]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
data:
  azure_storage_account_id: Y3Rja2FzdGVu
  azure_storage_key: d2p5RmsvZkRTMldrYUd3ZjhERUF2S2Q1L1ZsRjZIcXdLb2lHYWFIYjFtdWpLSTU1Sm1JaXRoMzFwWjVLejFmc2hsaFlRTEVCZld5ZWw4ejdBTmd5NHc9PQ==
kind: Secret
metadata:
  name: k10secret-dm9zg
  namespace: kasten-io
type: Opaque
EOF

secret/k10secret-dm9zg created


In [452]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
data:
  key: a2FzdGVuMXNjODhs
kind: Secret
metadata:
  name: k10-dr-secret
  namespace: kasten-io
type: Opaque
EOF

secret/k10-dr-secret created


In [453]:

cat << 'EOF' | kubectl apply -f -
apiVersion: config.kio.kasten.io/v1alpha1
kind: Profile
metadata:
  name: vsphere-ctc
  namespace: kasten-io
spec:
  infra:
    credential:
      secret:
        apiVersion: v1
        kind: secret
        name: k10secret-tblw6
        namespace: kasten-io
      secretType: VSphereKey
    type: VSphere
    vsphere:
      serverAddress: vc.container.demo.local
  type: Infra

EOF

profile.config.kio.kasten.io/vsphere-ctc created


In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
data:
  vsphere_password: UmF0aW5nZW4uMTIz
  vsphere_user: c2tvY2hAZGVtby5sb2NhbA==
kind: Secret
metadata:
  name: k10secret-tblw6
  namespace: kasten-io
type: Opaque

EOF

### change clusterID from above !!!

In [458]:
 #Install the helm chart that creates the K10 restore job and wait for completion of the `k10-restore` job
 #Assumes that K10 is installed in 'kasten-io' namespace.
 helm install k10-restore kasten/k10restore  \
    --set sourceClusterID=93212760-78fe-4b02-aee0-34ac59e46048 \
    --set profile.name=az-stephan \
    --set global.persistence.storageClass=vsphere-csi \
    --set skipResource="profiles" \
    --namespace=kasten-io 
    
#--set skipResource="profiles\,policies" \

NAME: k10-restore
LAST DEPLOYED: Wed Apr  7 15:34:07 2021
NAMESPACE: kasten-io
STATUS: deployed
REVISION: 1
TEST SUITE: None


![RestoreInProg](pictures/Restore-K10-inprog.png)

In [371]:
kubens kasten-io
kubectl get pod
helm list

Context "default" modified.
Active namespace is "kasten-io".
NAME                                  READY   STATUS      RESTARTS   AGE
aggregatedapis-svc-5cf667dbb8-gtzfn   1/1     Running     0          167m
dashboardbff-svc-66787557b5-fbvxx     1/1     Running     0          167m
auth-svc-54546c78ff-6bbbg             1/1     Running     0          167m
state-svc-5588b7845c-lrm9x            1/1     Running     0          167m
jobs-svc-76fc88c449-h4vgs             1/1     Running     0          167m
frontend-svc-784d5fb99c-rflzb         1/1     Running     0          167m
metering-svc-67fd96b56b-jxxbj         1/1     Running     0          167m
kanister-svc-8576d8bdfb-7fnct         1/1     Running     0          167m
executor-svc-6b8f7ff6-h2vr5           2/2     Running     0          167m
logging-svc-5d4bcc7f9d-x7rhv          1/1     Running     0          167m
executor-svc-6b8f7ff6-7fz7x           2/2     Running     0          167m
executor-svc-6b8f7ff6-ctx67           2/2     Runnin

#### recreate Infra Provider and then all your Applications